In [4]:
import json
import w3lib.html
import random
import os
from datetime import datetime
import requests
import string
import calendar
import html
import locale 
from requests_html import HTMLSession
from requests_html import HTML
import validators
import pandas as pd
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [5]:
URL_SEED = 'https://tierramarillano.cl/regionales/1/'
XPATH_HREF = "//h3/a/@href"

In [6]:
class Crawler():
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    #Ruta XPATH hacia los enlaces que queremos recopilar
    xpath_href=XPATH_HREF

    #variable que permite listar todos los enlaces recopilado
    url_list = []
    
    def _init_(self, URL_SEED, XPATH_HREF):        
        pass

    
    def get(self,url):
        session = HTMLSession()
        header = dict()
        header['user-agent'] = random.choice(self.USER_AGENT_LIST)
        response = session.get(url,headers=header)        
        return response
    
    def parse(self, response):
        self.url_list = []
        urls = response.html.xpath(self.xpath_href)
        for url in urls:
            self.url_list.append(url)


In [7]:
urls=[]

for page in tqdm(range(3)): # revisemos sólo 2 páginas
    crawler = Crawler()
    response = crawler.get('https://tierramarillano.cl/regionales/'+str(page))
    crawler.parse(response)

    
    for url in crawler.url_list:
        urls.append(url)

    crawler.url_list = urls
    
crawler.url_list

  0%|          | 0/3 [00:00<?, ?it/s]

['https://tierramarillano.cl/2022/04/03/chanaral-carabineros-del-os7-atacama-intercepto-millonario-cargamento-de-droga-con-destino-a-santiago/',
 'https://tierramarillano.cl/2022/04/03/fiscal-regional-propuestas-de-la-convencion-burocratizan-y-debilitan-la-persecucion-penal/',
 'https://tierramarillano.cl/2022/04/03/delegado-presidencial-gerardo-tapia-encabeza-reunion-con-secretarios-regionales-ministeriales-de-atacama/',
 'https://tierramarillano.cl/2022/04/03/directiva-del-capitulo-medico-del-hospital-de-vallenar-se-reune-con-delegado-presidencial-provincial-de-huasco/',
 'https://tierramarillano.cl/2022/04/03/codelco-salvador-lanza-programa-de-bienvenida-e-integracion-para-nuevos-trabajadores/',
 'https://tierramarillano.cl/2022/04/03/convenio-entre-gobierno-regional-y-nueva-atacama-permitira-acortar-tiempos-de-ejecucion-de-proyectos-habitacionales/',
 'https://tierramarillano.cl/2022/04/03/21-establecimientos-del-servicio-local-huasco-obtienen-la-excelencia-academica-2022-2023/',
 

In [8]:
len(urls)

36

In [9]:
urls[0]

'https://tierramarillano.cl/2022/04/03/chanaral-carabineros-del-os7-atacama-intercepto-millonario-cargamento-de-droga-con-destino-a-santiago/'

In [10]:
URL_NEWS = urls[0]
XPATH_TITLE = "//h1"
XPATH_CONTENT ="//div[@class='elementor-widget-container']//p"

In [11]:
class Scraper:
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    query_extract_title = XPATH_TITLE
    query_extract_text =  XPATH_CONTENT

    def _init_(self):
        pass
   
    def get(self,url):
        headers = {'user-agent':random.choice(self.USER_AGENT_LIST) }
        session = HTMLSession()
        response = session.get(url,headers=headers) 
        return response
    
    def clean_text(self, html_text):
        text_without_tags = w3lib.html.remove_tags(html_text)
        text_without_escape_chars = w3lib.html.replace_escape_chars(text_without_tags)
        text_without_escape_chars = html.unescape(text_without_escape_chars)
        text_without_whitespace = text_without_escape_chars.strip()
        return text_without_whitespace

    
    #### main function ####
    def parse(self, response):
        
        #URL
        self.news_url = response.url
        
        #TITLE
        self.news_title = response.html.xpath(self.query_extract_title)[0].text
        self.news_title = self.clean_text(self.news_title)
       
        
        #CONTENT
        self.news_text = ""
        
        html_paragraphs = response.html.xpath(self.query_extract_text)
        
        for paragraph in html_paragraphs:
            paragraph_content = w3lib.html.remove_tags(paragraph.text)    
            text_cleaned = self.clean_text(paragraph_content)
            self.news_text = self.news_text+text_cleaned


In [12]:
scraper = Scraper()
response =scraper.get(URL_NEWS)

In [13]:
scraper.parse(response)

In [14]:
scraper.news_title

'CHAÑARAL: CARABINEROS DEL OS7 ATACAMA INTERCEPTÓ MILLONARIO CARGAMENTO DE DROGA CON DESTINO A SANTIAGO'

In [15]:
scraper.news_text

'Una vez más el despliegue operativo de Carabineros de la Sección O.S.7 Atacama, dispuestos en la segunda línea de control, específicamente en la Ruta 5 Norte, Km. 986 de la comuna de Chañaral, permitió interceptar un millonario cargamento de drogas que tendría como destino final la Región Metropolitana.De acuerdo a los antecedentes entregados por el Jefe de la Sección OS7 Atacama, mayor Eugenio Olea García, esta importante cantidad de drogas era trasladada en el interior de un bolso y una maleta de un bus interprovincial, con itinerario Calama-Santiago, el cual al ser fiscalizado con el apoyo del ejemplar canino Frodo dejó al descubierto esta carga ilícita compuesta por 13 kilos 998 gramos de Marihuana y 1 Kilo 20 gramos de Pasta Base de Cocaína.“Es así como Carabineros a través de su permanente trabajo operativo y coordinado entre las unidades territoriales y de las secciones especializadas evita nuevamente que 34 mil dosis drogas lleguen a su destino”, destacó el mayor Olea.Tras rea

## LA NOTICIA PRESENTA BASURA EN SU CONTENT

In [16]:
#Verificamos que el scraper devuelve algo
assert(len(scraper.news_title)>0)
assert(len(scraper.news_text)>0)

In [17]:
df = pd.DataFrame(data={'url': [URL_NEWS],'title':[scraper.news_title], 'content':[scraper.news_text]})
df

,url,title,content
0,https://tierramarillano.cl/2022/04/03/chanaral...,CHAÑARAL: CARABINEROS DEL OS7 ATACAMA INTERCEP...,Una vez más el despliegue operativo de Carabin...


In [18]:
XPATH_TITLE = "//h1"
XPATH_CONTENT ="//div[@class='elementor-widget-container']//p"

for url in tqdm(urls):
    URL_NEWS = url
    scraper = Scraper()
    response =scraper.get(URL_NEWS)
    scraper.parse(response)
    df1 = pd.DataFrame(data={'url': url, 'title':scraper.news_title, 'content':scraper.news_text},index=['url', 'title', 'content'])
    df = df.append(df1, ignore_index=True)
df = df.drop_duplicates()


  0%|          | 0/36 [00:00<?, ?it/s]

In [19]:
df

,url,title,content
0,https://tierramarillano.cl/2022/04/03/chanaral...,CHAÑARAL: CARABINEROS DEL OS7 ATACAMA INTERCEP...,Una vez más el despliegue operativo de Carabin...
4,https://tierramarillano.cl/2022/04/03/fiscal-r...,Fiscal Regional: “propuestas de la Convención ...,Alexis Rogat Lucero compartió el planteamiento...
7,https://tierramarillano.cl/2022/04/03/delegado...,Delegado Presidencial Gerardo Tapia encabeza r...,"Este viernes, el Delegado Presidencial, Gerard..."
10,https://tierramarillano.cl/2022/04/03/directiv...,Directiva del Capítulo Médico del Hospital de ...,Una reunión entre la directiva del Capítulo Mé...
13,https://tierramarillano.cl/2022/04/03/codelco-...,Codelco Salvador lanza programa de bienvenida ...,Un programa de bienvenida e integración para l...
16,https://tierramarillano.cl/2022/04/03/convenio...,Convenio entre Gobierno Regional y Nueva Ataca...,La iniciativa transforma a la empresa sanitari...
19,https://tierramarillano.cl/2022/04/03/21-estab...,21 establecimientos del Servicio Local Huasco ...,Una gran noticia dio a conocer el Sistema Naci...
22,https://tierramarillano.cl/2022/04/02/jardines...,JARDINES INFANTILES DE CALDERA OBTIENEN SELLO ...,Con el objetivo de certificar el cumplimiento ...
25,https://tierramarillano.cl/2022/04/01/ricardo-...,Ricardo Zamora Hidalgo nuevo Seremi de Agricul...,El delegado presidencial Gerardo Tapia Tapia a...
28,https://tierramarillano.cl/2022/04/01/fiscalia...,Fiscalía sumó a dos imputados en investigación...,"Fiscal adjunto de Copiapó, Sebastián Coya Gonz..."
